In [6]:
import numpy as np

alpha=0.3 # capital share in production function
beta=0.6 # discount rate
A=20 # technology parameter in production function
K_max=10 # maximum of capital
grid=5 # numbers of grid points
max_iter=200 # maximum of iteration
tol=1e-4 # tolerance

# interpolate capital uniformly to the grid points
K=np.linspace(K_max, 0, grid, endpoint=False)[::-1]


def bellman(v, grid, beta):
    """
    bellman operator which transforms initial function v to Tv according to:
    
    Tv(k)=\max _{k'} \ln (A k^\alpha - k') +\beta* v(k')
    
    subject to:
    0\leq k' \leq A k^\alpha
    and 
    k' \in K
    ------
    parameters:
    v(k): initial value function
            ndarray, 1 \times grid
    grid: numbers of grids
            integer
    beta: discount factor
            float(dtype=32), 0<beta<1
    
    output:
    Tv(k): updated value function
            ndarray, 1 \times grid
    k'(k): control variable associated with Tv
            ndarray, 1 \times grid
    """
    # first, calculate the pseudo_v function:
    # \tilde v(k, k')=\ln (A*k^\alpha - k') +beta* v(k')
    #with the constraint: 
    # 0\leq k' \leq A*k^\alpha
    pseudo_v=np.zeros([grid, grid])
    for i in range(grid):
        k_prime_max=A*K[i]**alpha
        for j in range(grid):
            if K[j]<k_prime_max:
                pseudo_v[i, j]=np.log(A*K[i]**alpha-K[j])+beta*v[j]
            else:
                pseudo_v[i, j]=pseudo_v[i, j-1]
    # next, maximize pseudo_v row by row to get Tv(k) and k'(k)
    new_v=np.zeros(grid)
    K_prime=np.zeros(grid)
    K_prime_index=np.argmax(pseudo_v, axis=1)
    for i in range(grid):
        new_v[i]=max(pseudo_v[i])
        K_prime[i]=K[K_prime_index[i]]
    return new_v, K_prime

def update(v, N, grid, beta):
    """
    updates v according to the bellman operator for N times
    ------
    output:
    V: series of value functions
            ndarray, (N+1) \times grid
    K_p: series of control variables
            ndarray, N \times grid
    """
    V=np.zeros([N+1, grid])
    K_p=np.zeros([N, grid])
    V[0]=v
    for n in range(1, N+1):
        V[n], K_p[n-1]=bellman(V[n-1], grid, beta)
    return V, K_p

def vfi(v, grid, beta):
    """
    value function iteration until convergence or max_iter attained
    ------
    output:
    n: numbers of iteration
            integer
    V[n]: optimal value function
            ndarray, 1 \times grid
    K_p[n-1]: optimal policy function
            ndarray, 1 \times grid
    """
    V_before=v
    n=1
    while n<=max_iter+1:
        V_after, K_p=bellman(V_before, grid, beta)
        if max(abs(V_before-V_after))<=tol:
            return n, V_after, K_p
            break
        else:
            V_before=V_after
            n=n+1

# saving the optimal results
v_0=np.zeros(grid)
n_iter, V_opt, K_prime_opt=vfi(v_0, grid, beta)
sigma=[K, K_prime_opt]

def next_k(k):
    """
    calculate next period capital according to the optimal policy function
    """
    return sigma[1][np.where(abs(K-k)<0.01)]

def c_k_path(k, T):
    """
    derive the time path for consumption and capital for T periods
    """
    cc=np.zeros(T-1)
    kk=np.zeros(T)
    kk[0]=k
    for t in range(1, T):
        kk[t]=next_k(kk[t-1])
        cc[t-1]=np.log(A*kk[t-1]**alpha-kk[t])
    return cc, kk

In [7]:
#problem 4
update(v=v_0, N=3, grid=5, beta=0.6)

(array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
        [ 3.11896215,  3.34336808,  3.47306514,  3.56448674,  3.63508953],
        [ 5.03242239,  5.27613456,  5.42441549,  5.52813809,  5.60740884],
        [ 6.19208228,  6.44571524,  6.59522571,  6.6989483 ,  6.77965331]]),
 array([[ 2.,  2.,  2.,  2.,  2.],
        [ 4.,  4.,  6.,  6.,  6.],
        [ 4.,  6.,  6.,  6.,  8.]]))

In [5]:
z=update(v=v_0, N=3, grid=5, beta=0.6)[1]
np.savetxt("p4_2.csv", z, delimiter=",")

In [8]:
#problem 5
n_iter, V_opt, K_prime_opt

(22,
 array([ 7.94745286,  8.20182354,  8.35133401,  8.4550566 ,  8.53576161]),
 array([ 4.,  6.,  6.,  6.,  8.]))

In [9]:
zz=np.asarray([K, V_opt, K_prime_opt])
np.savetxt("p5.csv", zz, delimiter=",")

In [106]:
#problem 6
#change grid to 200
grid=200
beta=0.6
K=np.linspace(K_max, 0, grid, endpoint=False)[::-1]
v_0=np.zeros(grid)
n_iter1, V_opt1, K_prime_opt1=vfi(v_0, grid, beta)
sigma=[K, K_prime_opt1]
time1=np.arange(100)
c_path1, k_path1=c_k_path(7, 101)[0][0:100], c_k_path(7, 101)[1][0:100]

In [115]:
#problem 8
#change grid to 200
grid=200
#change beta to 0.75
beta=0.75
K=np.linspace(K_max, 0, grid, endpoint=False)[::-1]
v_0=np.zeros(grid)
n_iter2, V_opt2, K_prime_opt2=vfi(v_0, grid, beta)
sigma=[K, K_prime_opt2]
time1=np.arange(100)
c_path2, k_path2=c_k_path(7, 101)[0][0:100], c_k_path(7, 101)[1][0:100]

In [123]:
fig, ax=plt.subplots(2, 2, figsize=(7, 6))

ax[0, 0].plot(K, V_opt1, label=r'$\beta = 0.6$')
ax[0, 0].plot(K, V_opt2, label=r'$\beta = 0.75$', linestyle=':')
ax[0, 0].grid()
ax[0, 0].set_xlim(0, 9.99)
ax[0, 0].set_xlabel(r'$k$', fontsize=11)
ax[0, 0].set_ylabel(r'$\nu (k)$', fontsize=11)
ax[0, 0].legend(loc='lower right', fontsize=11)
ax[0, 0].spines['right'].set_visible(False)
ax[0, 0].spines['top'].set_visible(False)
ax[0, 0].set_title('value functions', fontsize=12.5, position=[.5, 1.02])
ax[0, 0].patch.set_facecolor('grey')
ax[0, 0].patch.set_alpha(0.06)

ax[0, 1].plot(K, K_prime_opt1, label=r'$\beta = 0.6$')
ax[0, 1].plot(K, K_prime_opt2, label=r'$\beta = 0.75$', linestyle=':')
ax[0, 1].grid()
ax[0, 1].set_xlim(0, 9.99)
ax[0, 1].set_xlabel(r'$k$', fontsize=11)
ax[0, 1].set_ylabel(r'$k^{\prime} (k)$', fontsize=11)
ax[0, 1].legend(loc='lower right', fontsize=11)
ax[0, 1].spines['right'].set_visible(False)
ax[0, 1].spines['top'].set_visible(False)
ax[0, 1].set_title('policy functions', fontsize=12.5, position=[.5, 1.02])
ax[0, 1].patch.set_facecolor('grey')
ax[0, 1].patch.set_alpha(0.06)

ax[1, 0].plot(time1, c_path1, label=r'$\beta = 0.6$')
ax[1, 0].plot(time1, c_path2, label=r'$\beta = 0.75$', linestyle=':')
ax[1, 0].grid()
ax[1, 0].set_xlim(0, 99)
ax[1, 0].set_xlabel(r'$t$', fontsize=11)
ax[1, 0].set_ylabel(r'$c_t$', fontsize=11)
ax[1, 0].legend(loc='lower right', fontsize=11)
ax[1, 0].spines['right'].set_visible(False)
ax[1, 0].spines['top'].set_visible(False)
ax[1, 0].set_title('consumption', fontsize=12.5, position=[.5, 1.02])
ax[1, 0].patch.set_facecolor('grey')
ax[1, 0].patch.set_alpha(0.06)

ax[1, 1].plot(time1, k_path1, label=r'$\beta = 0.6$')
ax[1, 1].plot(time1, k_path2, label=r'$\beta = 0.75$', linestyle=':')
ax[1, 1].grid()
ax[1, 1].set_xlim(0, 99)
ax[1, 1].set_xlabel(r'$t$', fontsize=11)
ax[1, 1].set_ylabel(r'$k_t$', fontsize=11)
ax[1, 1].legend(loc='lower right', fontsize=11)
ax[1, 1].spines['right'].set_visible(False)
ax[1, 1].spines['top'].set_visible(False)
ax[1, 1].set_title('capital', fontsize=12.5, position=[.5, 1.02])
ax[1, 1].patch.set_facecolor('grey')
ax[1, 1].patch.set_alpha(0.06)

plt.tight_layout()
plt.savefig('p8.pgf', dpi=300)

In [109]:
beta=0.6
def v_star(k):
    a_1=alpha/(1-alpha*beta)
    temp1=np.log(A)+alpha*beta*np.log(alpha*beta)+(1-alpha*beta)*np.log(1-alpha*beta)
    temp2=(1-alpha*beta)*(1-beta)
    a_0=temp1/temp2
    return a_0+a_1*np.log(k)

def k_prime_star(k):
    a_1=alpha/(1-alpha*beta)
    return k**alpha *A*beta*a_1/(1+beta*a_1)

v_true=np.zeros(grid)
k_prime_true=np.zeros(grid)
for i, k in enumerate(K):
    v_true[i]=v_star(k)
    k_prime_true[i]=k_prime_star(k)

v_true1, k_prime_true1 = v_true, k_prime_true

In [110]:
beta=0.75
def v_star(k):
    a_1=alpha/(1-alpha*beta)
    temp1=np.log(A)+alpha*beta*np.log(alpha*beta)+(1-alpha*beta)*np.log(1-alpha*beta)
    temp2=(1-alpha*beta)*(1-beta)
    a_0=temp1/temp2
    return a_0+a_1*np.log(k)

def k_prime_star(k):
    a_1=alpha/(1-alpha*beta)
    return k**alpha *A*beta*a_1/(1+beta*a_1)

v_true=np.zeros(grid)
k_prime_true=np.zeros(grid)
for i, k in enumerate(K):
    v_true[i]=v_star(k)
    k_prime_true[i]=k_prime_star(k)

v_true2, k_prime_true2 = v_true, k_prime_true

In [102]:
import matplotlib.pyplot as plt
fig, ax=plt.subplots(figsize=(5, 3))
ax.plot(K, v_true, color='red', lw=4, label=r'$\nu ^* (k)$'+': true value', linestyle=':', alpha=1)
ax.scatter(K, V_opt, color='blue', alpha=0.4,marker='o', s=12.5, label=r'$\nu _{\infty} (k)$'+': numerical result')

ax.grid()
ax.set_xlim(0, 9.99)
ax.set_ylim(6.5, 8.75)
ax.set_xlabel(r'$k$', fontsize=11)
ax.set_ylabel(r'$\nu (k)$', fontsize=11)
ax.legend(loc='lower right', fontsize=11)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_title('value functions when '+ r'$\alpha=0.3, \beta = 0.6$', fontsize=12, position=[.5, 1.02])
ax.patch.set_facecolor('grey')
ax.patch.set_alpha(0.06)
plt.savefig('p6.pgf', dpi=300)

In [105]:
import matplotlib.pyplot as plt
fig, ax=plt.subplots(figsize=(5, 3))
ax.plot(K, k_prime_true, color='red', lw=4, label=r'$k^{\prime *} (k)$'+': true value', linestyle=':', alpha=1)
ax.scatter(K, K_prime_opt, color='blue', alpha=0.4,marker='o', s=12.5, label=r'$k^{\prime} _{\infty} (k)$'+': numerical result')

ax.grid()
ax.set_xlim(0, 9.99)
ax.set_xlabel(r'$k$', fontsize=11)
ax.set_ylabel(r'$k^{\prime} (k)$' , fontsize=11)
ax.legend(loc='lower right', fontsize=11)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_title('policy functions when '+ r'$\alpha=0.3, \beta = 0.6$', fontsize=12, position=[.5, 1.02])
ax.patch.set_facecolor('grey')
ax.patch.set_alpha(0.06)
plt.savefig('p61.pgf', dpi=300)

In [21]:
np.shape(c_k_path(7, 101)[0][0:100])

(100,)

In [22]:
time1=np.arange(100)
p77, p78=c_k_path(7, 101)[0][0:100], c_k_path(7, 101)[1][0:100]
ok=np.asarray([time1, p77, p78])
np.savetxt("p7.csv", ok, delimiter=",")

In [23]:
#problem 7
c_k_path(7, 101)

(array([ 3.38119191,  3.35641936,  3.34955974,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.3

In [14]:
time1=np.arange(100)
time1

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [24]:
#problem 8
#change beta to 0.75
beta=0.75
K=np.linspace(K_max, 0, grid, endpoint=False)[::-1]
v_0=np.zeros(grid)
n_iter, V_opt, K_prime_opt=vfi(v_0, grid, beta)
sigma=[K, K_prime_opt]

xxx=np.asarray([K, V_opt, K_prime_opt])
np.savetxt("p81.csv", xxx, delimiter=",")

In [25]:
time1=np.arange(100)
p887, p888=c_k_path(7, 101)[0][0:100], c_k_path(7, 101)[1][0:100]
okk=np.asarray([time1, p887, p888])
np.savetxt("p80.csv", okk, delimiter=",")

In [20]:
#finally, the time path
c_k_path(7, 100)

(array([ 3.32524461,  3.36699042,  3.3800286 ,  3.38290805,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.3